<img src="Figs/GEOS_logo.pdf" width="500"/>

# Invert BC, InSAR and GNSS data (weighted LSM): <font color=blue>"inversion_BC_InSAR_GNSS_weighting.ipynb"</font>
#### Nov 18, 2021  <font color=red>(v. testing)</font>
##### Jeonghyeop Kim (jeonghyeop.kim@gmail.com)


>  input files :  \
> <font color=red> **Basis functions related to force terms** </font> \
> **`vel_hori_FT_i_j_on_InSAR.gmt`**, **`vel_hori_FT_i_j_on_boundary.gmt`**, **`vel_hori_FT_i_j_on_GNSS.gmt`** \
> **`vel_vert_FT_i_j_on_InSAR.gmt`**, **`vel_vert_FT_i_j_on_boundary.gmt`**, **`vel_vert_FT_i_j_on_GNSS.gmt`** \
> **`<i = 1..360 & j = 1..4>`** \
> <font color=red> **Basis functions related to boundary conditions** </font> \
**`vel_BC_k_l_on_boundary.gmt`**, **`vel_BC_k_l_on_InSAR.gmt`**, **`vel_BC_k_l_on_GNSS.gmt`** \
> **`<k = x, y, z & l = 001..038>`** \
> <font color=red> **Data vector made of GNSS (x and y), 2 InSAR data sets + Boundary velocities** </font> \
> **`DT42_data_new_ref_mm.dat`** and **`AT35_data_new_ref_mm.dat`** \
> **`<lon lat rate (mm/yr) Px Py Pz>`** \
> **`vel_GNSS_rotated.gmt`** : GNSS data (in mm/yr) \
> **`vel_BC_GNSS_ref_no_refpoint.gmt`** : B.C. velocities (in mm/yr)
>
> output files : \
> <font color=red> **predicted data** </font> \
> **`DT42_data_new_ref_pred.dat`** , **`AT35_data_new_ref_pred.dat`** \
> **`vel_GNSS_rotated_pred.gmt`** \
> **`vel_BC_pred.dat`** \
> **`vel_horizontal_cont_pred.gmt`** & **`vel_vertical_cont_pred.dat`** \
0. This code is a part of the joint inversion project (project4: joint inversion of GNSS and InSAR)
1. This code uses two types of basis functions generated in the previous steps : BC and FT 
2. This code jointly invert 4 InSAR data and 1 boundary velocity data for coefficients of each basis function
> (a) d=Gm (d=data; G=basis functions; m=model coefficients) \
> (b) m(best) = inv(G'G)G'd \
> (c) m(best) * continuous surface 3-D velocities (from basis functions) 
3. Try using different weighting factors for BC and GNSS data, and InSAR data.
> BC velocities and GNSS data have smaller error than InSAR data in general. \
> Apply the weighting LSM to the final step only. 
4. All "magic" comments were removed due to errors when a converted script is used. 

In [1]:
#import numpy and pandas
import numpy as np
import pandas as pd
import scipy 
import sys

In [2]:
# # # # If you use jupyter notebook, 
# # # # comment out this cell and use the cell below instead. 

# weight_for_BC = sys.argv[1]
# weight_for_BC = float(weight_for_BC)

# weight_for_GNSS = weight_for_BC

# weight_for_InSAR = sys.argv[2]
# weight_for_InSAR = float(weight_for_InSAR)

In [3]:
# # If you use the converted python script,
# # comment out this cell and use the cell above instead.
weight_for_BC = 1
weight_for_GNSS = weight_for_BC
weight_for_InSAR = 8 # This weighting factor will be inversely taken e.g., 8 => 1/8

In [4]:
## Inversion types

#inversion_flag = 1 # Simple LSM
inversion_flag = 2 # Pseudo LSM 
#inversion_flag = 3 # Damped LSM

### `STEP1`: **BUILD a data vector,  $\vec{d}$**

In [5]:
# load input files

# 1. Boundary velocity (48 data points on the boundary)
inputBC = "vel_BC_GNSS_ref_no_refpoint.gmt"  #velocity boundary condition
df_inputBC = pd.read_csv(inputBC, header = None, sep =' ')
df_inputBC.columns = ['lon','lat','ve','vn','se','sn','corr']
df_inputBC.loc[:,['ve']] = df_inputBC.loc[:,['ve']]  
df_inputBC.loc[:,['vn']] = df_inputBC.loc[:,['vn']]

# 2. GNSS data
inputGNSS = "vel_GNSS_rotated.gmt"  # GNSS
df_inputGNSS = pd.read_csv(inputGNSS, header = None, sep=r'(?:,|\s+)', comment='#', engine='python')
df_inputGNSS.columns = ['lon','lat','ve','vn','se','sn','corr']
df_inputGNSS.loc[:,['ve']] = df_inputGNSS.loc[:,['ve']] 
df_inputGNSS.loc[:,['vn']] = df_inputGNSS.loc[:,['vn']]

# 2. InSAR Descending 
inputInSAR1 = "DT42_data_new_ref_mm.dat"
df_inputInSAR1 = pd.read_csv(inputInSAR1, header = None, sep = ' ')
df_inputInSAR1.columns = ['lon','lat','velo','Px','Py','Pz']  
df_inputInSAR1.loc[:,['velo']] = df_inputInSAR1.loc[:,['velo']]

# 3. InSAR Ascending 
inputInSAR2 = "AT35_data_new_ref_mm.dat"
df_inputInSAR2 = pd.read_csv(inputInSAR2, header = None, sep = ' ')
df_inputInSAR2.columns = ['lon','lat','velo','Px','Py','Pz'] 
df_inputInSAR2.loc[:,['velo']] = df_inputInSAR2.loc[:,['velo']]

<div class="alert alert-success">
<b>NOTE: the pointing vectors are from the perspective of the ground! NOT of the satellite </b> 
</div>

In [6]:
# BUILD a boundary condition data vector along with coordinate information.
# The x components are first and then the y components of the velocity.

df_data_x = df_inputGNSS.iloc[:,[0,1,2]]  # saved vx data on the boudnary
df_data_y = df_inputGNSS.iloc[:,[0,1,3]]  # saved vn data on the boundary

df_data_x=df_data_x.rename(columns ={'ve': 'velo'}) #column name change
df_data_y=df_data_y.rename(columns ={'vn': 'velo'}) #column name change

# !! SORT_VALUES !! # lat ascending first, and then lon ascending.
# This step is very important to build the G matrix, G, which
# has rows correspoding to the rows of the data vector, d, that have
# the same coordinates!

df_data_x=df_data_x.sort_values(['lat', 'lon'], ascending=[True, True])
df_data_y=df_data_y.sort_values(['lat', 'lon'], ascending=[True, True])

# MERGE two columns (n*1) into a new column (2n*1)
# > ignore_index = True : 
# >   have one continuous index numbers,
# >     ignorning each of the two dfs original indices

framesGNSS=[df_data_x,df_data_y]
df_data_GNSS_all=pd.concat(framesGNSS,ignore_index=True) # merge the two dataFrames into one

df_data_GNSS=df_data_GNSS_all.loc[:,['velo']]

In [7]:
# BUILD a boundary condition data vector along with coordinate information.
# The x components are first and then the y components of the velocity.

df_data_x = df_inputBC.iloc[:,[0,1,2]]  # saved vx data on the boudnary
df_data_y = df_inputBC.iloc[:,[0,1,3]]  # saved vn data on the boundary

df_data_x=df_data_x.rename(columns ={'ve': 'velo'}) #column name change
df_data_y=df_data_y.rename(columns ={'vn': 'velo'}) #column name change

# !! SORT_VALUES !! # lat ascending first, and then lon ascending.
# This step is very important to build the G matrix, G, which
# has rows correspoding to the rows of the data vector, d, that have
# the same coordinates!

df_data_x=df_data_x.sort_values(['lat', 'lon'], ascending=[True, True])
df_data_y=df_data_y.sort_values(['lat', 'lon'], ascending=[True, True])

# MERGE two columns (n*1) into a new column (2n*1)
# > ignore_index = True : 
# >   have one continuous index numbers,
# >     ignorning each of the two dfs original indices

framesBC=[df_data_x,df_data_y]
df_data_BC_all=pd.concat(framesBC,ignore_index=True) # merge the two dataFrames into one

df_data_BC=df_data_BC_all.loc[:,['velo']]


In [8]:
# BUILD a InSAR data vector along with coordinate information.
# The rows of the InSAR data vector is in the order of Ascending 1, Asceding 2, Descending 1, and Descending 2. 
# Track the pointing vector values together with the rate data for the G-matrix.

df_inputInSAR1=df_inputInSAR1.sort_values(['lat', 'lon'], ascending=[True, True])
df_inputInSAR2=df_inputInSAR2.sort_values(['lat', 'lon'], ascending=[True, True])


framesInSAR=[df_inputInSAR1,df_inputInSAR2]
df_data_InSAR_all=pd.concat(framesInSAR,ignore_index=True) # merge the four dataFrames into one
df_data_InSAR = df_data_InSAR_all.loc[:,['velo']]

In [9]:
# Merge the GNSS, InSAR, and BC data vectors into the final data vector

framesFinal = [df_data_GNSS, df_data_InSAR, df_data_BC]
df_data_total = pd.concat(framesFinal,ignore_index=True) # merge the two dataFrames into one

In [10]:
df_data_total.columns = ['data'] # DATA VECTOR [GNSSx; GNSSy; InSAR1; InSAR2; BCx; BCy]

NOTE:   
>If your data have errors, \
build a **W** matrix in which its diagonal elements \
are of a vector made of 1/errors (in order).  \
Your new data vector **d<sub>w</sub>** = **W** **d**

### `STEP2`: **BUILD G-Matrix, $\bar{\bar{G}}$**

> Comment on the read_csv(sep option) below: \
> **`'(?:,|\s+)'`** - is a RegEx for selecting either comma or any number of consecutive spaces/tabs \
> See the answer at StackOverFlow [click here](https://stackoverflow.com/questions/43784422/pandas-error-when-reading-csv-file-using-sep-and-comment-arguments)

`STEP 2-a : Build G matrix part only related to Boundary Condition on GNSS data points`

In [11]:
df_G_BC_on_GNSS = pd.DataFrame(index = range(len(df_data_GNSS))) 
# Make a blank G matrix part related to Boundary Condition on GNSS points

# print('How many files do you have for the boudnary basis functions? :')
# HowMany = input()
# HowMany = int(HowMany)
#38 for the boundary condition (11/17/2021)
HowMany=38
# print("38 (fixed for now [11/17/2021])")

for i in range(1,HowMany+1): 

    inputfile_xrot = "vel_BC_x_"+str(f"{i:03}")+"_on_GNSS.gmt" # x-rot 
    inputfile_yrot = "vel_BC_y_"+str(f"{i:03}")+"_on_GNSS.gmt" # y-rot 
    inputfile_zrot = "vel_BC_z_"+str(f"{i:03}")+"_on_GNSS.gmt" # z-rot 

# READ files in order {xrot1, yrot1, zrot1, ..., xrotHowMany, yrotHowMany, zrotHowMany}

    df_xrot=pd.read_csv(inputfile_xrot ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_yrot=pd.read_csv(inputfile_yrot ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_zrot=pd.read_csv(inputfile_zrot ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')

# CHANGE the column names 

    df_xrot.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_yrot.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_zrot.columns = ['lon','lat','ve','vn','se','sn','corr']
    
# BUILD a column vector Gx (i)

    df_xrot_x = df_xrot.iloc[:,[0,1,2]]  # saved vx basis function on the boudnary
    df_xrot_y = df_xrot.iloc[:,[0,1,3]]  # saved vn basis function on the boundary

    df_xrot_x=df_xrot_x.rename(columns ={'ve': 'velo'}) #column name change
    df_xrot_y=df_xrot_y.rename(columns ={'vn': 'velo'}) #column name change

#     df_xrot_x['flag']=np.array([1] * len(df_xrot_x)) # 1 = velocity east-component
#     df_xrot_y['flag']=np.array([2] * len(df_xrot_y)) # 2 = velocity north-component
    
    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_xrot_x=df_xrot_x.sort_values(['lat', 'lon'], ascending=[True, True])
    df_xrot_y=df_xrot_y.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices   
    frames_Gx=[df_xrot_x,df_xrot_y]
    df_Gx=pd.concat(frames_Gx,ignore_index=True) # merge the two dataFrames into one

    
    
# BUILD a column vector Gy (i)

    df_yrot_x = df_yrot.iloc[:,[0,1,2]]  # saved vx basis function on the GNSS points
    df_yrot_y = df_yrot.iloc[:,[0,1,3]]  # saved vn basis function on the GNSS points

    df_yrot_x=df_yrot_x.rename(columns ={'ve': 'velo'}) #column name change
    df_yrot_y=df_yrot_y.rename(columns ={'vn': 'velo'}) #column name change

#     df_yrot_x['flag']=np.array([1] * len(df_yrot_x)) # 1 = velocity east-component
#     df_yrot_y['flag']=np.array([2] * len(df_yrot_y)) # 2 = velocity north-component

    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_yrot_x=df_yrot_x.sort_values(['lat', 'lon'], ascending=[True, True])
    df_yrot_y=df_yrot_y.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Gy=[df_yrot_x,df_yrot_y]
    df_Gy=pd.concat(frames_Gy,ignore_index=True) # merge the two dataFrames into one
    
    
# BUILD a column vector Gz (i)

    df_zrot_x = df_zrot.iloc[:,[0,1,2]]  # saved vx basis function on the GNSS points
    df_zrot_y = df_zrot.iloc[:,[0,1,3]]  # saved vn basis function on the GNSS points

    df_zrot_x=df_zrot_x.rename(columns ={'ve': 'velo'}) #column name change
    df_zrot_y=df_zrot_y.rename(columns ={'vn': 'velo'}) #column name change

#     df_zrot_x['flag']=np.array([1] * len(df_zrot_x)) # 1 = velocity east-component
#     df_zrot_y['flag']=np.array([2] * len(df_zrot_y)) # 2 = velocity north-component
   
    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_zrot_x=df_zrot_x.sort_values(['lat', 'lon'], ascending=[True, True])
    df_zrot_y=df_zrot_y.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Gz=[df_zrot_x,df_zrot_y]
    df_Gz=pd.concat(frames_Gz,ignore_index=True) # merge the two dataFrames into one
    
    
# SAVE G-matrix
# Gmatrix = [Gxrot(1) Gyrot(1) Gzrot(1) ... Gxrot(HowMany) Gyrot(HowMany) Gzrot(HowMany)]
    
    df_G_BC_on_GNSS["G_xrot"+str(i)] = df_Gx.loc[:,['velo']]
    df_G_BC_on_GNSS["G_yrot"+str(i)] = df_Gy.loc[:,['velo']]
    df_G_BC_on_GNSS["G_zrot"+str(i)] = df_Gz.loc[:,['velo']]
    
#df_G_BC_on_GNSS

`STEP 2-b : Build G matrix part related to both Boundary Condition and Force Terms on GNSS data points`

In [12]:
df_G_FT_on_GNSS_eij = pd.DataFrame(index = range(len(df_data_GNSS))) 
df_G_FT_on_GNSS_ezz = pd.DataFrame(index = range(len(df_data_GNSS))) 
# Make a blank G matrix part related to Boundary Condition on GNSS data points

# print('How many grid cells do you have (where a set of 4 force terms defined) ? :')
HowMany=360
# print("360 (fixed for now [9/29/2021])")

for i in range(1,HowMany+1): 

    inputfile_exx = "vel_hori_FT_"+str(i)+"_1"+"_on_GNSS.gmt" #exx horizontal
    inputfile_eyy = "vel_hori_FT_"+str(i)+"_2"+"_on_GNSS.gmt" #eyy horizontal
    inputfile_exy = "vel_hori_FT_"+str(i)+"_3"+"_on_GNSS.gmt" #exy horizontal 
    inputfile_ezz = "vel_hori_FT_"+str(i)+"_4"+"_on_GNSS.gmt" # z  horizontal
    
## READ files into two separate structures in the following orders:
## 1st stru = {exx1, eyy1, exy1, ..., exxHowMany, eyyHowMany, exyHowMany} 
## 2nd stru = {z1 .. zHowMany}
##
## And then merge these two structures into one in the order of ...
##            {exx1, eyy1, exy1, ..., exxHowMany, eyyHowMany, exyHowMany, z1 .. zHowMany}


    df_exx=pd.read_csv(inputfile_exx ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_eyy=pd.read_csv(inputfile_eyy ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_exy=pd.read_csv(inputfile_exy ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')   
    df_ezz=pd.read_csv(inputfile_ezz, header=None, sep=r'(?:,|\s+)',
                           comment='#', engine='python')

# CHANGE the column names 

    df_exx.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_eyy.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_exy.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_ezz.columns = ['lon','lat','ve','vn','se','sn','corr']
    
# BUILD a column vector Gexx (i)

    df_exx_x = df_exx.iloc[:,[0,1,2]]  # saved vx basis function on the GNSS points
    df_exx_y = df_exx.iloc[:,[0,1,3]]  # saved vn basis function on the GNSS points

    df_exx_x=df_exx_x.rename(columns ={'ve': 'velo'}) #column name change
    df_exx_y=df_exx_y.rename(columns ={'vn': 'velo'}) #column name change
    
    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_exx_x=df_exx_x.sort_values(['lat', 'lon'], ascending=[True, True])
    df_exx_y=df_exx_y.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices   
    frames_Gexx=[df_exx_x,df_exx_y]
    df_Gexx=pd.concat(frames_Gexx,ignore_index=True) # merge the two dataFrames into one

    
# BUILD a column vector Geyy (i)

    df_eyy_x = df_eyy.iloc[:,[0,1,2]]  # saved vx basis function on the GNSS points
    df_eyy_y = df_eyy.iloc[:,[0,1,3]]  # saved vn basis function on the GNSS points

    df_eyy_x=df_eyy_x.rename(columns ={'ve': 'velo'}) #column name change
    df_eyy_y=df_eyy_y.rename(columns ={'vn': 'velo'}) #column name change

    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_eyy_x=df_eyy_x.sort_values(['lat', 'lon'], ascending=[True, True])
    df_eyy_y=df_eyy_y.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Geyy=[df_eyy_x,df_eyy_y]
    df_Geyy=pd.concat(frames_Geyy,ignore_index=True) # merge the two dataFrames into one
    
    
# BUILD a column vector Gexy (i)

    df_exy_x = df_exy.iloc[:,[0,1,2]]  # saved vx basis function on the GNSS points
    df_exy_y = df_exy.iloc[:,[0,1,3]]  # saved vn basis function on the GNSS points

    df_exy_x=df_exy_x.rename(columns ={'ve': 'velo'}) #column name change
    df_exy_y=df_exy_y.rename(columns ={'vn': 'velo'}) #column name change

   
    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_exy_x=df_exy_x.sort_values(['lat', 'lon'], ascending=[True, True])
    df_exy_y=df_exy_y.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Gexy=[df_exy_x,df_exy_y]
    df_Gexy=pd.concat(frames_Gexy,ignore_index=True) # merge the two dataFrames into one


# BUILD a column vector G_ezz (i)

    df_ezz_x = df_ezz.iloc[:,[0,1,2]]  # saved vx basis function on the GNSS points
    df_ezz_y = df_ezz.iloc[:,[0,1,3]]  # saved vn basis function on the GNSS points

    df_ezz_x=df_ezz_x.rename(columns ={'ve': 'velo'}) #column name change
    df_ezz_y=df_ezz_y.rename(columns ={'vn': 'velo'}) #column name change

   
    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_ezz_x=df_ezz_x.sort_values(['lat', 'lon'], ascending=[True, True])
    df_ezz_y=df_ezz_y.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Gezz=[df_ezz_x,df_ezz_y]
    df_Gezz=pd.concat(frames_Gezz,ignore_index=True) # merge the two dataFrames into one (vertically)
    
    
    
# SAVE a part of G-matrix (as in two different structures and then they will be merged later)

    # 1st structure = [Gexx(1) Geyy(1) Gexy(1) ... Gexx(HowMany) Geyy(HowMany) Gexy(HowMany)]   
    df_G_FT_on_GNSS_eij["G_exx"+str(i)] = df_Gexx.loc[:,['velo']]
    df_G_FT_on_GNSS_eij["G_eyy"+str(i)] = df_Geyy.loc[:,['velo']]
    df_G_FT_on_GNSS_eij["G_exy"+str(i)] = df_Gexy.loc[:,['velo']]

    # 2nd structure = [Gezz(1) Gezz(2) ... Gezz(HowMany)] 
    df_G_FT_on_GNSS_ezz["G_ezz"+str(i)] = df_Gezz.loc[:,['velo']]
    
    
# Merge the two structures horizontally !

frames_Geij_Gezz = [df_G_FT_on_GNSS_eij, df_G_FT_on_GNSS_ezz]
df_G_FT_on_GNSS=pd.concat(frames_Geij_Gezz, axis=1) # merge the two dataFrames into one

`STEP 2-c : Merge FT and BC basis functions horizontally. This G-matrix is for GNSS data points`

In [13]:
# Merge FT and BC basis functions horizontally !
frames_FT_BC=[df_G_FT_on_GNSS, df_G_BC_on_GNSS]
df_G_FT_BC_on_GNSS = pd.concat(frames_FT_BC, axis=1) #merge the two dataFrames into onee horizontally (axis = 1)

`STEP 2-d : Build G matrix part only related to Boundary Condition on boundary points`

In [14]:
df_G_BC_on_boundary = pd.DataFrame(index = range(len(df_data_BC))) 
# Make a blank G matrix part related to Boundary Condition on boundary points

# print('How many files do you have for the boudnary basis functions? :')
# HowMany = input()
# HowMany = int(HowMany)
#38 for the boundary condition (11/16/2021)
HowMany=38
# print("38 (fixed for now [11/16/2021])")

for i in range(1,HowMany+1): 

    inputfile_xrot = "vel_BC_x_"+str(f"{i:03}")+"_on_boundary.gmt" # x-rot 
    inputfile_yrot = "vel_BC_y_"+str(f"{i:03}")+"_on_boundary.gmt" # y-rot 
    inputfile_zrot = "vel_BC_z_"+str(f"{i:03}")+"_on_boundary.gmt" # z-rot 

# READ files in order {xrot1, yrot1, zrot1, ..., xrotHowMany, yrotHowMany, zrotHowMany}

    df_xrot=pd.read_csv(inputfile_xrot ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_yrot=pd.read_csv(inputfile_yrot ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_zrot=pd.read_csv(inputfile_zrot ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')

# CHANGE the column names 

    df_xrot.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_yrot.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_zrot.columns = ['lon','lat','ve','vn','se','sn','corr']
    
# BUILD a column vector Gx (i)

    df_xrot_x = df_xrot.iloc[:,[0,1,2]]  # saved vx basis function on the boudnary
    df_xrot_y = df_xrot.iloc[:,[0,1,3]]  # saved vn basis function on the boundary

    df_xrot_x=df_xrot_x.rename(columns ={'ve': 'velo'}) #column name change
    df_xrot_y=df_xrot_y.rename(columns ={'vn': 'velo'}) #column name change

#     df_xrot_x['flag']=np.array([1] * len(df_xrot_x)) # 1 = velocity east-component
#     df_xrot_y['flag']=np.array([2] * len(df_xrot_y)) # 2 = velocity north-component
    
    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_xrot_x=df_xrot_x.sort_values(['lat', 'lon'], ascending=[True, True])
    df_xrot_y=df_xrot_y.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices   
    frames_Gx=[df_xrot_x,df_xrot_y]
    df_Gx=pd.concat(frames_Gx,ignore_index=True) # merge the two dataFrames into one

    
    
# BUILD a column vector Gy (i)

    df_yrot_x = df_yrot.iloc[:,[0,1,2]]  # saved vx basis function on the boudnary
    df_yrot_y = df_yrot.iloc[:,[0,1,3]]  # saved vn basis function on the boundary

    df_yrot_x=df_yrot_x.rename(columns ={'ve': 'velo'}) #column name change
    df_yrot_y=df_yrot_y.rename(columns ={'vn': 'velo'}) #column name change

#     df_yrot_x['flag']=np.array([1] * len(df_yrot_x)) # 1 = velocity east-component
#     df_yrot_y['flag']=np.array([2] * len(df_yrot_y)) # 2 = velocity north-component

    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_yrot_x=df_yrot_x.sort_values(['lat', 'lon'], ascending=[True, True])
    df_yrot_y=df_yrot_y.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Gy=[df_yrot_x,df_yrot_y]
    df_Gy=pd.concat(frames_Gy,ignore_index=True) # merge the two dataFrames into one
    
    
# BUILD a column vector Gz (i)

    df_zrot_x = df_zrot.iloc[:,[0,1,2]]  # saved vx basis function on the boudnary
    df_zrot_y = df_zrot.iloc[:,[0,1,3]]  # saved vn basis function on the boundary

    df_zrot_x=df_zrot_x.rename(columns ={'ve': 'velo'}) #column name change
    df_zrot_y=df_zrot_y.rename(columns ={'vn': 'velo'}) #column name change

#     df_zrot_x['flag']=np.array([1] * len(df_zrot_x)) # 1 = velocity east-component
#     df_zrot_y['flag']=np.array([2] * len(df_zrot_y)) # 2 = velocity north-component
   
    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_zrot_x=df_zrot_x.sort_values(['lat', 'lon'], ascending=[True, True])
    df_zrot_y=df_zrot_y.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Gz=[df_zrot_x,df_zrot_y]
    df_Gz=pd.concat(frames_Gz,ignore_index=True) # merge the two dataFrames into one
    
    
# SAVE G-matrix
# Gmatrix = [Gxrot(1) Gyrot(1) Gzrot(1) ... Gxrot(HowMany) Gyrot(HowMany) Gzrot(HowMany)]
    
    df_G_BC_on_boundary["G_xrot"+str(i)] = df_Gx.loc[:,['velo']]
    df_G_BC_on_boundary["G_yrot"+str(i)] = df_Gy.loc[:,['velo']]
    df_G_BC_on_boundary["G_zrot"+str(i)] = df_Gz.loc[:,['velo']]
    
#df_G_BC_on_boundary

`STEP 2-e : Build G matrix part related to both Boundary Condition and Force Terms on boundary points`

In [15]:
df_G_FT_on_boundary_eij = pd.DataFrame(index = range(len(df_data_BC))) 
df_G_FT_on_boundary_ezz = pd.DataFrame(index = range(len(df_data_BC))) 
# Make a blank G matrix part related to Boundary Condition on boundary points

# print('How many grid cells do you have (where a set of 4 force terms defined) ? :')
HowMany=360
# print("360 (fixed for now [11/16/2021])")

for i in range(1,HowMany+1): 

    inputfile_exx = "vel_hori_FT_"+str(i)+"_1"+"_on_boundary.gmt" #exx horizontal
    inputfile_eyy = "vel_hori_FT_"+str(i)+"_2"+"_on_boundary.gmt" #eyy horizontal
    inputfile_exy = "vel_hori_FT_"+str(i)+"_3"+"_on_boundary.gmt" #exy horizontal 
    inputfile_ezz = "vel_hori_FT_"+str(i)+"_4"+"_on_boundary.gmt" # z  horizontal
    
## READ files into two separate structures in the following orders:
## 1st stru = {exx1, eyy1, exy1, ..., exxHowMany, eyyHowMany, exyHowMany} 
## 2nd stru = {z1 .. zHowMany}
##
## And then merge these two structures into one in the order of ...
##            {exx1, eyy1, exy1, ..., exxHowMany, eyyHowMany, exyHowMany, z1 .. zHowMany}


    df_exx=pd.read_csv(inputfile_exx ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_eyy=pd.read_csv(inputfile_eyy ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_exy=pd.read_csv(inputfile_exy ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')   
    df_ezz=pd.read_csv(inputfile_ezz, header=None, sep=r'(?:,|\s+)',
                           comment='#', engine='python')

# CHANGE the column names 

    df_exx.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_eyy.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_exy.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_ezz.columns = ['lon','lat','ve','vn','se','sn','corr']
    
# BUILD a column vector Gexx (i)

    df_exx_x = df_exx.iloc[:,[0,1,2]]  # saved vx basis function on the boudnary
    df_exx_y = df_exx.iloc[:,[0,1,3]]  # saved vn basis function on the boundary

    df_exx_x=df_exx_x.rename(columns ={'ve': 'velo'}) #column name change
    df_exx_y=df_exx_y.rename(columns ={'vn': 'velo'}) #column name change
    
    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_exx_x=df_exx_x.sort_values(['lat', 'lon'], ascending=[True, True])
    df_exx_y=df_exx_y.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices   
    frames_Gexx=[df_exx_x,df_exx_y]
    df_Gexx=pd.concat(frames_Gexx,ignore_index=True) # merge the two dataFrames into one

    
# BUILD a column vector Geyy (i)

    df_eyy_x = df_eyy.iloc[:,[0,1,2]]  # saved vx basis function on the boudnary
    df_eyy_y = df_eyy.iloc[:,[0,1,3]]  # saved vn basis function on the boundary

    df_eyy_x=df_eyy_x.rename(columns ={'ve': 'velo'}) #column name change
    df_eyy_y=df_eyy_y.rename(columns ={'vn': 'velo'}) #column name change

    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_eyy_x=df_eyy_x.sort_values(['lat', 'lon'], ascending=[True, True])
    df_eyy_y=df_eyy_y.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Geyy=[df_eyy_x,df_eyy_y]
    df_Geyy=pd.concat(frames_Geyy,ignore_index=True) # merge the two dataFrames into one
    
    
# BUILD a column vector Gexy (i)

    df_exy_x = df_exy.iloc[:,[0,1,2]]  # saved vx basis function on the boudnary
    df_exy_y = df_exy.iloc[:,[0,1,3]]  # saved vn basis function on the boundary

    df_exy_x=df_exy_x.rename(columns ={'ve': 'velo'}) #column name change
    df_exy_y=df_exy_y.rename(columns ={'vn': 'velo'}) #column name change

   
    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_exy_x=df_exy_x.sort_values(['lat', 'lon'], ascending=[True, True])
    df_exy_y=df_exy_y.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Gexy=[df_exy_x,df_exy_y]
    df_Gexy=pd.concat(frames_Gexy,ignore_index=True) # merge the two dataFrames into one


# BUILD a column vector G_ezz (i)

    df_ezz_x = df_ezz.iloc[:,[0,1,2]]  # saved vx basis function on the boudnary
    df_ezz_y = df_ezz.iloc[:,[0,1,3]]  # saved vn basis function on the boundary

    df_ezz_x=df_ezz_x.rename(columns ={'ve': 'velo'}) #column name change
    df_ezz_y=df_ezz_y.rename(columns ={'vn': 'velo'}) #column name change

   
    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_ezz_x=df_ezz_x.sort_values(['lat', 'lon'], ascending=[True, True])
    df_ezz_y=df_ezz_y.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Gezz=[df_ezz_x,df_ezz_y]
    df_Gezz=pd.concat(frames_Gezz,ignore_index=True) # merge the two dataFrames into one (vertically)
    
    
    
# SAVE a part of G-matrix (as in two different structures and then they will be merged later)

    # 1st structure = [Gexx(1) Geyy(1) Gexy(1) ... Gexx(HowMany) Geyy(HowMany) Gexy(HowMany)]   
    df_G_FT_on_boundary_eij["G_exx"+str(i)] = df_Gexx.loc[:,['velo']]
    df_G_FT_on_boundary_eij["G_eyy"+str(i)] = df_Geyy.loc[:,['velo']]
    df_G_FT_on_boundary_eij["G_exy"+str(i)] = df_Gexy.loc[:,['velo']]

    # 2nd structure = [Gezz(1) Gezz(2) ... Gezz(HowMany)] 
    df_G_FT_on_boundary_ezz["G_ezz"+str(i)] = df_Gezz.loc[:,['velo']]
    
    
# Merge the two structures horizontally !

frames_Geij_Gezz = [df_G_FT_on_boundary_eij, df_G_FT_on_boundary_ezz]
df_G_FT_on_boundary=pd.concat(frames_Geij_Gezz, axis=1) # merge the two dataFrames into one
#df_G_FT_on_boundary

`STEP 2-f : Merge FT and BC basis functions horizontally. This G-matrix is for boundary velocity points`

In [16]:
# Merge FT and BC basis functions horizontally !
frames_FT_BC=[df_G_FT_on_boundary, df_G_BC_on_boundary]
df_G_FT_BC_on_boundary = pd.concat(frames_FT_BC, axis=1) #merge the two dataFrames into onee horizontally (axis = 1)
#df_G_FT_BC_on_boundary

`STEP 2-g : Build G matrix part only related to Force Terms on InSAR data points`

In [17]:
df_G_FT_on_InSAR_hori = pd.DataFrame(index = range(len(df_data_InSAR))) 
df_G_FT_on_InSAR_vert = pd.DataFrame(index = range(len(df_data_InSAR))) 
# Make a blank G matrix part related to Force Terms on InSAR data points

# df_G_FT_on_InSAR :
# will be 'concat'ed with a frame of [df_G_FT_on_InSAR_hori, df_G_FT_on_InSAR_vert] and with axis=1

# df_data_InSAR : velocity only
# df_data_InSAR_all : lon lat vel px py pz

df_px = df_data_InSAR_all.iloc[:,[3]]
df_py = df_data_InSAR_all.iloc[:,[4]]
df_pz = df_data_InSAR_all.iloc[:,[5]]

#print('How many grid cells do you have (where a set of 4 force terms defined) ? :'
HowMany=360
#print("360 (fixed for now [11/16/2021])")

for i in range(1,HowMany+1): 

    inputfile_exx = "vel_hori_FT_"+str(i)+"_1"+"_on_InSAR.gmt" #exx horizontal
    inputfile_eyy = "vel_hori_FT_"+str(i)+"_2"+"_on_InSAR.gmt" #eyy horizontal
    inputfile_exy = "vel_hori_FT_"+str(i)+"_3"+"_on_InSAR.gmt" #exy horizontal 
    inputfile_ezz = "vel_vert_FT_"+str(i)+"_4"+"_on_InSAR.gmt" # z  vertical
    
## READ files into two separate structures in the following orders:
## 1st stru = {exx1_x*px + exx1_y*py, eyy1_x*px + eyy1_y*py , exy1_x*px + exy1_y*px, ..., 
##             eyyHowMany_x*px + eyyHowMany_y*py, exyHowMany_x*px + exyHowMany_y*py} 
## 2nd stru = {ezz1_z*pz, ezz2_z*pz, ..., ezzHowMany_z*pz}
##
## And then merge these two structures into one in the order of ...
##            {exx1_x*px + exx1_y*py,...,exyHowMany_x*px + exyHowMany_y*py,ezz1_z*pz, ..., ezzHowMany_z*pz}


    df_exx=pd.read_csv(inputfile_exx ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_eyy=pd.read_csv(inputfile_eyy ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_exy=pd.read_csv(inputfile_exy ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')   
    df_ezz=pd.read_csv(inputfile_ezz, header=None, sep=r'(?:,|\s+)',
                           comment='#', engine='python')

# CHANGE the column names 

    # ve = Px
    # vn = Py
    # vz = Pz 
    # This is because later on ...
    # 've' will be multiplied by a dataFrame named 'Px'
    # 'vn' will be multiplied by a dataFrame named 'Py'
    # 'vz' will be multiplied by a dataFrame named 'Pz'
    # pandas does Not allow to multiply (element wise) two different df in different names
    
    df_exx.columns = ['lon','lat','Px','Py','se','sn','corr']
    df_eyy.columns = ['lon','lat','Px','Py','se','sn','corr']
    df_exy.columns = ['lon','lat','Px','Py','se','sn','corr']
    df_ezz.columns = ['lon','lat','Pz']

# SORT THEM by the same order of the InSAR data,
# which was already sorted in the beginning of this code.
    
    
    df_exx=df_exx.sort_values(['lat', 'lon'], ascending=[True, True])
    df_eyy=df_eyy.sort_values(['lat', 'lon'], ascending=[True, True])
    df_exy=df_exy.sort_values(['lat', 'lon'], ascending=[True, True])
    df_ezz=df_ezz.sort_values(['lat', 'lon'], ascending=[True, True])

# STACK the data frames of the basis function responses vertically!
# InSAR data vector made of 2 separate pointing vectors
# but the 2 data sets are defined in the same coordinates. 
# One needs to stack 2 of df_eij vertically. 
# len(df_eij) = len(InSAR_data) / 2

# len(df_eij_stacked) = len(InSAR_data) !!
     
    
    frames_exx_stack = [df_exx,df_exx] # Two insar data sets!
    df_exx_stacked=pd.concat(frames_exx_stack,ignore_index=True) 
    # merge the two dataFrames into one

    frames_eyy_stack = [df_eyy,df_eyy] # Two insar data sets!
    df_eyy_stacked=pd.concat(frames_eyy_stack,ignore_index=True) 
    # merge the two dataFrames into one    

    frames_exy_stack = [df_exy,df_exy] # Two insar data sets!
    df_exy_stacked=pd.concat(frames_exy_stack,ignore_index=True) 
    # merge the two dataFrames into one

    frames_ezz_stack = [df_ezz,df_ezz] # Two insar data sets!
    df_ezz_stacked=pd.concat(frames_ezz_stack,ignore_index=True) 
    # merge the two dataFrames into one

    
    
# # BUILD a column vector Gx (i)
    df_LOS_Gexx_x = df_exx_stacked.loc[:,['Px']] * df_px 
    df_LOS_Gexx_y = df_exx_stacked.loc[:,['Py']] * df_py
    df_LOS_Gexx_x.columns=['Py'] # pandas doesn't add columns with different names directly
    df_LOS_Gexx = df_LOS_Gexx_x + df_LOS_Gexx_y
    
    
    df_LOS_Geyy_x = df_eyy_stacked.loc[:,['Px']] * df_px 
    df_LOS_Geyy_y = df_eyy_stacked.loc[:,['Py']] * df_py
    df_LOS_Geyy_x.columns=['Py'] # pandas doesn't add columns with different names directly
    df_LOS_Geyy = df_LOS_Geyy_x + df_LOS_Geyy_y
    
    
    df_LOS_Gexy_x = df_exy_stacked.loc[:,['Px']] * df_px 
    df_LOS_Gexy_y = df_exy_stacked.loc[:,['Py']] * df_py
    df_LOS_Gexy_x.columns=['Py'] # pandas doesn't add columns with different names directly
    df_LOS_Gexy = df_LOS_Gexy_x + df_LOS_Gexy_y

    
    df_LOS_Gezz = df_ezz_stacked.loc[:,['Pz']] * df_pz
    

# SAVE G-matrix
# df_G_FT_on_InSAR_hori = [(exx1_x*px + exx1_y*py), ... , (exyHowMany_x*px + exyHowMany_y*py)]

    
    df_G_FT_on_InSAR_hori["GexxXPx+GexxYPy "+str(i)] = df_LOS_Gexx.loc[:,['Py']]
    df_G_FT_on_InSAR_hori["GeyyXPx+GeyyYPy "+str(i)] = df_LOS_Geyy.loc[:,['Py']]
    df_G_FT_on_InSAR_hori["GexyXPx+GexyYPy "+str(i)] = df_LOS_Gexy.loc[:,['Py']]
    
    df_G_FT_on_InSAR_vert["GezzZPz "+str(i)] = df_LOS_Gezz['Pz']
    
    
frames_FT_InSAR=[df_G_FT_on_InSAR_hori, df_G_FT_on_InSAR_vert]
df_G_FT_on_InSAR = pd.concat(frames_FT_InSAR, axis=1) #merge the two dataFrames into onee horizontally (axis = 1)

`STEP 2-h : Build G matrix part related to both Force Terms and Boundary Conditions on InSAR data points`

In [18]:
df_G_BC_on_InSAR = pd.DataFrame(index = range(len(df_data_InSAR))) 
# Make a blank G matrix part related to Bounndary Condition on InSAR data points

# df_G_BC_on_InSAR :

# df_data_InSAR : velocity only
# df_data_InSAR_all : lon lat vel px py pz

df_px = df_data_InSAR_all.iloc[:,[3]]
df_py = df_data_InSAR_all.iloc[:,[4]]
df_pz = df_data_InSAR_all.iloc[:,[5]]

#print('How many files do you have for the boudnary basis functions? :')
#38 for the boundary condition (11/16/2021)
HowMany=38
#print("38 (fixed for now [11/16/2021])")

for i in range(1,HowMany+1): 

    inputfile_xrot = "vel_BC_x_"+str(f"{i:03}")+"_on_InSAR.gmt" # x-rot 
    inputfile_yrot = "vel_BC_y_"+str(f"{i:03}")+"_on_InSAR.gmt" # y-rot 
    inputfile_zrot = "vel_BC_z_"+str(f"{i:03}")+"_on_InSAR.gmt" # z-rot 

# READ files in order {xrot1, yrot1, zrot1, ..., xrotHowMany, yrotHowMany, zrotHowMany}

    df_xrot=pd.read_csv(inputfile_xrot ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_yrot=pd.read_csv(inputfile_yrot ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_zrot=pd.read_csv(inputfile_zrot ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')

# CHANGE the column names 
    # ve = Px
    # vn = Py
    # vz = Pz 
    # This is because later on ...
    # 've' will be multiplied by a dataFrame named 'Px'
    # 'vn' will be multiplied by a dataFrame named 'Py'
    # 'vz' will be multiplied by a dataFrame named 'Pz'
    # pandas does Not allow to multiply (element wise) two different df in different names

    df_xrot.columns = ['lon','lat','Px','Py','se','sn','corr']  
    df_yrot.columns = ['lon','lat','Px','Py','se','sn','corr']
    df_zrot.columns = ['lon','lat','Px','Py','se','sn','corr']
    

    
# SORT THEM by the same order of the InSAR data,
# which was already sorted in the beginning of this code.

    df_xrot=df_xrot.sort_values(['lat', 'lon'], ascending=[True, True])
    df_yrot=df_yrot.sort_values(['lat', 'lon'], ascending=[True, True])
    df_zrot=df_zrot.sort_values(['lat', 'lon'], ascending=[True, True])

# STACK the data frames of the basis function responses vertically!
# InSAR data vector made of 2 separate pointing vectors
# but the 2 data sets are defined in the same coordinates. 
# One needs to stack 2 of df_irot vertically. 
# len(df_xrot) = len(InSAR_data)/2 
    
# len(df_xrot_stacked) = len(InSAR_data) !!
    
    frames_xrot_stack = [df_xrot,df_xrot]
    df_xrot_stacked=pd.concat(frames_xrot_stack,ignore_index=True) # merge the two dataFrames into one

    
    frames_yrot_stack = [df_yrot,df_yrot]
    df_yrot_stacked=pd.concat(frames_yrot_stack,ignore_index=True) # merge the two dataFrames into one
    
    
    frames_zrot_stack = [df_zrot,df_zrot]
    df_zrot_stacked=pd.concat(frames_zrot_stack,ignore_index=True) # merge the two dataFrames into one
    
    
    
# # BUILD a column vector Gx (i)
    df_LOS_Gxrot_x = df_xrot_stacked.loc[:,['Px']] * df_px 
    df_LOS_Gxrot_y = df_xrot_stacked.loc[:,['Py']] * df_py
    df_LOS_Gxrot_x.columns=['Py'] # pandas doesn't add columns with different names directly
    df_LOS_Gxrot = df_LOS_Gxrot_x + df_LOS_Gxrot_y
    

    df_LOS_Gyrot_x = df_yrot_stacked.loc[:,['Px']] * df_px 
    df_LOS_Gyrot_y = df_yrot_stacked.loc[:,['Py']] * df_py
    df_LOS_Gyrot_x.columns=['Py'] # pandas doesn't add columns with different names directly
    df_LOS_Gyrot = df_LOS_Gyrot_x + df_LOS_Gyrot_y
    

    df_LOS_Gzrot_x = df_zrot_stacked.loc[:,['Px']] * df_px 
    df_LOS_Gzrot_y = df_zrot_stacked.loc[:,['Py']] * df_py
    df_LOS_Gzrot_x.columns=['Py'] # pandas doesn't add columns with different names directly
    df_LOS_Gzrot = df_LOS_Gzrot_x + df_LOS_Gzrot_y
    

# SAVE G-matrix
# df_G_FT_on_InSAR_hori = [(xrot1_x*px + xrot1_y*py), ... , (zrotHowMany_x*px + zrotHowMany_y*py)]

    
    df_G_BC_on_InSAR["GxrotXPx+GxrotYPy "+str(i)] = df_LOS_Gxrot.loc[:,['Py']]
    df_G_BC_on_InSAR["GyrotXPx+GyrotYPy "+str(i)] = df_LOS_Gyrot.loc[:,['Py']]
    df_G_BC_on_InSAR["GzrotXPx+GzrotYPy "+str(i)] = df_LOS_Gzrot.loc[:,['Py']]
    

#df_G_BC_on_InSAR

`STEP 2-i : Merge FT and BC basis functions horizontally. This G-matrix is for InSAR points`

In [19]:
# Merge FT and BC basis functions on InSAR data point horizontally !
frames_FT_BC_on_InSAR=[df_G_FT_on_InSAR, df_G_BC_on_InSAR]
df_G_FT_BC_on_InSAR = pd.concat(frames_FT_BC_on_InSAR, axis=1) #merge the two dataFrames into onee horizontally (axis = 1)
#df_G_FT_BC_on_InSAR

# `STEP 2-FINAL : Build the complete G matrix `
    
### G-matrix = [df_G_FT_BC_on_GNSS; df_G_FT_BC_on_InSAR; df_G_FT_BC_on_boundary]


In [20]:
df_G_FT_BC_on_boundary.columns = df_G_FT_BC_on_InSAR.columns
df_G_FT_BC_on_GNSS.columns = df_G_FT_BC_on_InSAR.columns
# change column names of the df_G_FT_BC_on_boundary
# change column names of the df_G_FT_BC_on_GNSS

frames_FT_BC_final=[df_G_FT_BC_on_GNSS, df_G_FT_BC_on_InSAR, df_G_FT_BC_on_boundary]
df_G_final = pd.concat(frames_FT_BC_final, ignore_index=True) #merge the two dataFrames into one vertically (axis = 1)
#df_G_final

In [21]:
if len(df_data_total)!=len(df_G_final):
    print("WARNING: Something went wrong!")

# `**STEP3**` : Joint Inversion of GNSS, InSAR data and Boundary velocity field
> G-matrix = **df_G_final** \
> data vec = **df_data_total**

In [22]:
# Build a Diagonal Weighting Matrix W
nGNSS=len(df_data_GNSS)
nBC=len(df_data_BC)
nInSAR=len(df_data_InSAR)
nTotal=len(df_data_total)

errorGNSS = np.ones(nGNSS)*weight_for_GNSS
errorInSAR = np.ones(nInSAR)*weight_for_InSAR 
errorBC = np.ones(nBC)*weight_for_BC

errorTotal = np.concatenate((errorGNSS,errorInSAR, errorBC),axis=0)
errorTotalinv = 1/errorTotal
W = np.diag(errorTotalinv)

# convert into a dataframe
dfW = pd.DataFrame(W)

# When calculating predictions, the non-weighted G-matrix is needed. 
# SAVE it!
df_G_final_save = df_G_final

# When calculating the misfit, the non-weighted data is needed. 
# SAVE it!
df_data_total_save = df_data_total

In [23]:
# Multiply the Diagonal Weighting Matrix dfW to the data vector and Gmatrix

df_G_final = dfW @ df_G_final_save
df_data_total = dfW @ df_data_total_save

In [24]:
df_G_prime = df_G_final.transpose() 

# G'G
# >Two different ways to compute a matrix multiplication
# >1st method
GpG1=df_G_prime.dot(df_G_final) #G'G
# >2nd method
GpG2=df_G_prime @ df_G_final #G'G
# >These results are same.
# >Let's take the second one as G'G
GpG = GpG2 #GpG is G'G
# inv(G'G)
# > Two different ways to obtain inverse matrix
# > 1st method: np.linalg.inv 
df_inv_GpG = pd.DataFrame(np.linalg.inv(GpG.to_numpy()), GpG.columns, GpG.index)
# > 2nd method: np.linalg.pinv (Moore-Penrose inverse (SVD))
df_pinv_GpG = pd.DataFrame(np.linalg.pinv(GpG.to_numpy()), GpG.columns, GpG.index)

###############################
## inv(G'G)*G'*d = model(LSM) #
###############################
if inversion_flag == 1:
    df_model1=df_inv_GpG@df_G_prime@df_data_total #inversion
else:
    df_model1=df_pinv_GpG@df_G_prime@df_data_total #pseudo inversion

## data predicted.
#df_data_predicted = df_G_final@df_model1 
df_data_predicted = df_G_final_save @ df_model1
# df_G_final_save is the non-weighted G-matrix


## norm2 misfit
df_norm2=(df_data_total_save-df_data_predicted)**2
df_norm2=df_norm2.to_numpy().sum()
#df_norm2=np.lib.scimath.sqrt(df_norm2)


In [25]:
# Separate the predicted data vector into
# (1) GNSS x and y
# (2) InSAR 1
# (3) InSAR 2
# (4) BC vel x and y

num_velo_point0=len(df_data_GNSS_all) # GNSS
num_velo_point0=int(num_velo_point0)

num_velo_point1=len(df_data_InSAR_all)/2  # 2 InSAR data sets in a column vector
num_velo_point1=int(num_velo_point1)

num_velo_point2=len(df_data_BC_all) #BC vel
num_velo_point2=int(num_velo_point2)



####################
#####  GNSS  #######
####################

#GNSS predicted x
df_prediction_GNSS_x=df_data_predicted.iloc[0:int(num_velo_point0/2)] 
df_prediction_GNSS_x=df_prediction_GNSS_x.reset_index(drop=True)
#BC predicted y
df_prediction_GNSS_y=df_data_predicted.iloc[int(num_velo_point0/2):num_velo_point0] 
df_prediction_GNSS_y=df_prediction_GNSS_y.reset_index(drop=True)

# Coordinate Template To Save Predicted BC Data
df_GNSS_save = df_data_GNSS_all.iloc[0:int(num_velo_point0/2),[0,1]]

df_save_GNSS_XandY = df_GNSS_save.reset_index(drop=True)
df_save_GNSS_XandY['vx'] = df_prediction_GNSS_x
df_save_GNSS_XandY['vn'] = df_prediction_GNSS_y
df_save_GNSS_XandY['se'] = np.zeros(len(df_prediction_GNSS_y))
df_save_GNSS_XandY['sn'] = np.zeros(len(df_prediction_GNSS_y))
df_save_GNSS_XandY['corr'] = np.zeros(len(df_prediction_GNSS_y))



#####################
#####  INSAR  #######
#####################

df_prediction_D=df_data_predicted.iloc[num_velo_point0:num_velo_point0+num_velo_point1] #dLOS Descending
df_prediction_D=df_prediction_D.reset_index(drop=True)

df_prediction_A=df_data_predicted.iloc[num_velo_point0+num_velo_point1:num_velo_point0+2*num_velo_point1] #dLOS Ascending
df_prediction_A=df_prediction_A.reset_index(drop=True)

# Coordinate Template To Save Predicted InSAR Data
df_InSAR_save = df_inputInSAR1.iloc[:,[0,1]]   

df_save_D = df_InSAR_save.reset_index(drop=True)
df_save_D['dLOS'] = df_prediction_D #append predicted dLOS Descending
df_save_D['dLOS'] = df_save_D['dLOS']

df_save_A = df_InSAR_save.reset_index(drop=True)
df_save_A['dLOS'] = df_prediction_A #append predicted dLOS Ascending
df_save_A['dLOS'] = df_save_A['dLOS']

########################
#####  Boundary ########
########################

#BC predicted x
df_prediction_BC_x=df_data_predicted.iloc[num_velo_point0+2*num_velo_point1:num_velo_point0+2*num_velo_point1+int(num_velo_point2/2)] 
df_prediction_BC_x=df_prediction_BC_x.reset_index(drop=True)
#BC predicted y
df_prediction_BC_y=df_data_predicted.iloc[num_velo_point0+2*num_velo_point1+int(num_velo_point2/2):num_velo_point0+2*num_velo_point1+num_velo_point2] 
df_prediction_BC_y=df_prediction_BC_y.reset_index(drop=True)
# Coordinate Template To Save Predicted BC Data
df_BC_save = df_data_BC_all.iloc[0:int(num_velo_point2/2),[0,1]]
df_save_BC_XandY = df_BC_save.reset_index(drop=True)
df_save_BC_XandY['vx'] = df_prediction_BC_x
df_save_BC_XandY['vn'] = df_prediction_BC_y
df_save_BC_XandY['se'] = np.zeros(len(df_prediction_BC_y))
df_save_BC_XandY['sn'] = np.zeros(len(df_prediction_BC_y))
df_save_BC_XandY['corr'] = np.zeros(len(df_prediction_BC_y))


In [26]:
#########################################################################################
#########################################################################################
#################################   SAVE FILES       ####################################
#########################################################################################
#########################################################################################
#########################################################################################
#  SAVE predicted GNSS velocity values on the GNSS data points in ...
#  **`vel_GNSS_rotated_pred.gmt`**

#  SAVE predicted InSAR velocity values on the InSAR data points in ...                     
#  **`DT42_data_new_ref_pred.dat`** &
#  **`AT35_data_new_ref_pred.dat`**

#  SAVE predicted BC velocity values on the boundary data points in ...
#  **`vel_BC_pred.dat`**
#########################################################################################


#df_data_GNSS_all
#df_data_InSAR_all
#df_data_BC_all


outputFILE_GNSS_XandY="vel_GNSS_rotated_pred.gmt"
df_save_GNSS_XandY.to_csv(outputFILE_GNSS_XandY, header=None, index=None, sep=' ', float_format='%g')

outputFILE_D="DT42_data_new_ref_pred.dat"
df_save_D.to_csv(outputFILE_D, header=None, index=None, sep=' ',float_format='%g')

outputFILE_A="AT35_data_new_ref_pred.dat"
df_save_A.to_csv(outputFILE_A, header=None, index=None, sep=' ',float_format='%g')

outputFILE_BC_XandY="vel_BC_pred.dat"
df_save_BC_XandY.to_csv(outputFILE_BC_XandY, header=None, index=None, sep=' ', float_format='%g')

<div class="alert alert-warning">
<div class="alert--icon"> <i class="far fa-times-circle"></i> </div>
    <p> Simple LSM is not able to recover BC velocity well. Try weighted inversion </p>
</div>

<div class="alert alert-success">
    <p> The weighted LSM works well. </p>
    <b> There is tradeoff between data sets near the edges though. </b>
</div>

In [32]:
print("chi-square statistics is : %f [mm/yr]**2 when the weighting factor for InSAR and GNSS were %i and %i, respectively"  % (df_norm2, weight_for_InSAR,weight_for_GNSS))

chi-square statistics is : 40013.447177 [mm/yr]**2 when the weighting factor for InSAR and GNSS were 8 and 1, respectively


In [28]:
outputFILE_model="model_coef.dat"
df_model1.to_csv(outputFILE_model, header=None, index=None, float_format='%g')

# **`STEP 4`** : Get the corresponding 3-D model field using the model coefficients

In [29]:
########################################################################################
########################################################################################
#                          Force Balance Equation                                      #
########################################################################################
########################################################################################

knotpoints_sample = np.loadtxt('vel_hori_FT_1_1_on_knotpoints.gmt')
knotpoints_XandY=len(knotpoints_sample)*2
knotpoints_Z=len(knotpoints_sample)

df_G_FT_on_knotpoints_eij = pd.DataFrame(index = range(knotpoints_XandY)) 
df_G_FT_on_knotpoints_ezz = pd.DataFrame(index = range(knotpoints_XandY)) 
df_G_FT_on_knotpoints_zzz = pd.DataFrame(index = range(knotpoints_Z))
# Make a blank G matrix part related to Boundary Condition on boundary points

# print('How many grid cells do you have (where a set of 4 force terms defined) ? :')
HowMany=360
# print("360 (fixed for now [11/16/2021])")

for i in range(1,HowMany+1): 

    inputfile_exx = "vel_hori_FT_"+str(i)+"_1"+"_on_knotpoints.gmt" #exx horizontal
    inputfile_eyy = "vel_hori_FT_"+str(i)+"_2"+"_on_knotpoints.gmt" #eyy horizontal
    inputfile_exy = "vel_hori_FT_"+str(i)+"_3"+"_on_knotpoints.gmt" #exy horizontal 
    inputfile_ezz = "vel_hori_FT_"+str(i)+"_4"+"_on_knotpoints.gmt" # z  horizontal   
    inputfile_zzz = "vel_vert_FT_"+str(i)+"_4"+"_on_knotpoints.gmt" # z  vertical

    df_exx=pd.read_csv(inputfile_exx ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_eyy=pd.read_csv(inputfile_eyy ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_exy=pd.read_csv(inputfile_exy ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')   
    df_ezz=pd.read_csv(inputfile_ezz, header=None, sep=r'(?:,|\s+)',
                           comment='#', engine='python')
    df_zzz=pd.read_csv(inputfile_zzz, header=None, sep=r'(?:,|\s+)',
                           comment='#', engine='python')
    
# CHANGE the column names 

    df_exx.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_eyy.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_exy.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_ezz.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_zzz.columns = ['lon','lat','vz']
    
# BUILD a column vector Gexx (i)

    df_exx_x = df_exx.iloc[:,[0,1,2]]  # saved vx basis function on the knotpoints
    df_exx_y = df_exx.iloc[:,[0,1,3]]  # saved vn basis function on the knotpoints

    df_exx_x=df_exx_x.rename(columns ={'ve': 'velo'}) #column name change
    df_exx_y=df_exx_y.rename(columns ={'vn': 'velo'}) #column name change
    
    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_exx_x=df_exx_x.sort_values(['lat', 'lon'], ascending=[True, True])
    df_exx_y=df_exx_y.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices   
    frames_Gexx=[df_exx_x,df_exx_y]
    df_Gexx=pd.concat(frames_Gexx,ignore_index=True) # merge the two dataFrames into one

    
# BUILD a column vector Geyy (i)

    df_eyy_x = df_eyy.iloc[:,[0,1,2]]  # saved vx basis function on the knotpoints
    df_eyy_y = df_eyy.iloc[:,[0,1,3]]  # saved vn basis function on the knotpoints

    df_eyy_x=df_eyy_x.rename(columns ={'ve': 'velo'}) #column name change
    df_eyy_y=df_eyy_y.rename(columns ={'vn': 'velo'}) #column name change

    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_eyy_x=df_eyy_x.sort_values(['lat', 'lon'], ascending=[True, True])
    df_eyy_y=df_eyy_y.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Geyy=[df_eyy_x,df_eyy_y]
    df_Geyy=pd.concat(frames_Geyy,ignore_index=True) # merge the two dataFrames into one
    
    
# BUILD a column vector Gexy (i)

    df_exy_x = df_exy.iloc[:,[0,1,2]]  # saved vx basis function on the knotpoints
    df_exy_y = df_exy.iloc[:,[0,1,3]]  # saved vn basis function on the knotpoints

    df_exy_x=df_exy_x.rename(columns ={'ve': 'velo'}) #column name change
    df_exy_y=df_exy_y.rename(columns ={'vn': 'velo'}) #column name change

   
    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_exy_x=df_exy_x.sort_values(['lat', 'lon'], ascending=[True, True])
    df_exy_y=df_exy_y.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Gexy=[df_exy_x,df_exy_y]
    df_Gexy=pd.concat(frames_Gexy,ignore_index=True) # merge the two dataFrames into one


# BUILD a column vector G_ezz (i)

    df_ezz_x = df_ezz.iloc[:,[0,1,2]]  # saved vx basis function on the knotpoints
    df_ezz_y = df_ezz.iloc[:,[0,1,3]]  # saved vn basis function on the knotpoints

    df_ezz_x=df_ezz_x.rename(columns ={'ve': 'velo'}) #column name change
    df_ezz_y=df_ezz_y.rename(columns ={'vn': 'velo'}) #column name change

   
    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_ezz_x=df_ezz_x.sort_values(['lat', 'lon'], ascending=[True, True])
    df_ezz_y=df_ezz_y.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Gezz=[df_ezz_x,df_ezz_y]
    df_Gezz=pd.concat(frames_Gezz,ignore_index=True) # merge the two dataFrames into one (vertically)
    
# BUILD a column vector G_zzz (i)   
    df_zzz=df_zzz.sort_values(['lat','lon'], ascending=[True, True])
    df_zzz=df_zzz.reset_index(drop=True)
# SAVE a part of G-matrix (as in two different structures and then they will be merged later)

    # 1st structure = [Gexx(1) Geyy(1) Gexy(1) ... Gexx(HowMany) Geyy(HowMany) Gexy(HowMany)]   
    df_G_FT_on_knotpoints_eij["G_exx"+str(i)] = df_Gexx.loc[:,['velo']]
    df_G_FT_on_knotpoints_eij["G_eyy"+str(i)] = df_Geyy.loc[:,['velo']]
    df_G_FT_on_knotpoints_eij["G_exy"+str(i)] = df_Gexy.loc[:,['velo']]

    # 2nd structure = [Gezz(1) Gezz(2) ... Gezz(HowMany)] 
    df_G_FT_on_knotpoints_ezz["G_ezz"+str(i)] = df_Gezz.loc[:,['velo']]
    df_G_FT_on_knotpoints_zzz["G_zzz"+str(i)] = df_zzz.loc[:,['vz']]
    
# Merge the two structures horizontally !
frames_Geij_Gezz = [df_G_FT_on_knotpoints_eij, df_G_FT_on_knotpoints_ezz]
df_G_FT_on_knotpoints=pd.concat(frames_Geij_Gezz, axis=1) # merge the two dataFrames into one
#df_G_FT_on_knotpoints #horizontal
#df_G_FT_on_knotpoints_zzz #vertical

########################################################################################
########################################################################################
#                          Boundary Rotation velo                                      #
########################################################################################
########################################################################################

df_G_BC_on_knotpoints = pd.DataFrame(index = range(knotpoints_XandY)) 
# Make a blank G matrix part related to Boundary Condition on the knotpoints

# print('How many files do you have for the boudnary basis functions? :')
# HowMany = input()
# HowMany = int(HowMany)
#38 for the boundary condition (11/16/2021)
HowMany=38
# print("38 (fixed for now [11/16/2021])")

for i in range(1,HowMany+1): 

    inputfile_xrot = "vel_BC_x_"+str(f"{i:03}")+"_on_knotpoints.gmt" # x-rot 
    inputfile_yrot = "vel_BC_y_"+str(f"{i:03}")+"_on_knotpoints.gmt" # y-rot 
    inputfile_zrot = "vel_BC_z_"+str(f"{i:03}")+"_on_knotpoints.gmt" # z-rot 

# READ files in order {xrot1, yrot1, zrot1, ..., xrotHowMany, yrotHowMany, zrotHowMany}

    df_xrot=pd.read_csv(inputfile_xrot ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_yrot=pd.read_csv(inputfile_yrot ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_zrot=pd.read_csv(inputfile_zrot ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')

# CHANGE the column names 

    df_xrot.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_yrot.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_zrot.columns = ['lon','lat','ve','vn','se','sn','corr']
    
# BUILD a column vector Gx (i)

    df_xrot_x = df_xrot.iloc[:,[0,1,2]]  # saved vx basis function on the knotpoints
    df_xrot_y = df_xrot.iloc[:,[0,1,3]]  # saved vn basis function on the knotpoints

    df_xrot_x=df_xrot_x.rename(columns ={'ve': 'velo'}) #column name change
    df_xrot_y=df_xrot_y.rename(columns ={'vn': 'velo'}) #column name change
    
    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_xrot_x=df_xrot_x.sort_values(['lat', 'lon'], ascending=[True, True])
    df_xrot_y=df_xrot_y.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices   
    frames_Gx=[df_xrot_x,df_xrot_y]
    df_Gx=pd.concat(frames_Gx,ignore_index=True) # merge the two dataFrames into one

    
# BUILD a column vector Gy (i)

    df_yrot_x = df_yrot.iloc[:,[0,1,2]]  # saved vx basis function on the knotpoints
    df_yrot_y = df_yrot.iloc[:,[0,1,3]]  # saved vn basis function on the knotpoints

    df_yrot_x=df_yrot_x.rename(columns ={'ve': 'velo'}) #column name change
    df_yrot_y=df_yrot_y.rename(columns ={'vn': 'velo'}) #column name change

    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_yrot_x=df_yrot_x.sort_values(['lat', 'lon'], ascending=[True, True])
    df_yrot_y=df_yrot_y.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Gy=[df_yrot_x,df_yrot_y]
    df_Gy=pd.concat(frames_Gy,ignore_index=True) # merge the two dataFrames into one
    
    
# BUILD a column vector Gz (i)

    df_zrot_x = df_zrot.iloc[:,[0,1,2]]  # saved vx basis function on the knotpoints
    df_zrot_y = df_zrot.iloc[:,[0,1,3]]  # saved vn basis function on the knotpoints

    df_zrot_x=df_zrot_x.rename(columns ={'ve': 'velo'}) #column name change
    df_zrot_y=df_zrot_y.rename(columns ={'vn': 'velo'}) #column name change

    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_zrot_x=df_zrot_x.sort_values(['lat', 'lon'], ascending=[True, True])
    df_zrot_y=df_zrot_y.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Gz=[df_zrot_x,df_zrot_y]
    df_Gz=pd.concat(frames_Gz,ignore_index=True) # merge the two dataFrames into one
    
    
# SAVE G-matrix
# Gmatrix = [Gxrot(1) Gyrot(1) Gzrot(1) ... Gxrot(HowMany) Gyrot(HowMany) Gzrot(HowMany)]
    
    df_G_BC_on_knotpoints["G_xrot"+str(i)] = df_Gx.loc[:,['velo']]
    df_G_BC_on_knotpoints["G_yrot"+str(i)] = df_Gy.loc[:,['velo']]
    df_G_BC_on_knotpoints["G_zrot"+str(i)] = df_Gz.loc[:,['velo']]
    
#df_G_BC_on_knotpoints




########################################################################################
########################################################################################
#                         Horizontal continuous velocity model.                        #
########################################################################################
########################################################################################

frames_horizontal = [df_G_FT_on_knotpoints, df_G_BC_on_knotpoints]
df_G_horizontal_continuous=pd.concat(frames_horizontal, axis=1) # merge the two dataFrames into one


continuous_hor_model = df_G_horizontal_continuous.to_numpy() @ df_model1.to_numpy()
knotpointnum=int(len(continuous_hor_model)/2)
Xmodel=continuous_hor_model[0:knotpointnum,0]
Ymodel=continuous_hor_model[knotpointnum:,0]

########################################################################################
########################################################################################
#                         Vertical continuous velocity model.                          #
########################################################################################
########################################################################################

df_G_vertical_continuous=df_G_FT_on_knotpoints_zzz

continuous_ver_model = df_G_vertical_continuous.to_numpy() @ df_model1.to_numpy()[360*3:360*4,]
Zmodel=continuous_ver_model[:,0]




In [30]:
# SAVE continuous field.

In [31]:
df_hori = df_zzz.loc[:,['lon','lat']]
df_hori['ve'] = Xmodel
df_hori['vn'] = Ymodel
df_hori['se'] = np.zeros(len(Ymodel))
df_hori['sn'] = np.zeros(len(Ymodel))
df_hori['corr'] = np.zeros(len(Ymodel))

df_vert = df_zzz.loc[:,['lon','lat']]
df_vert['vz'] = Zmodel


outputFILE_hori="vel_horizontal_cont_pred.gmt"
df_hori.to_csv(outputFILE_hori, header=None, index=None, sep=' ', float_format='%g')

outputFILE_vert="vel_vertical_cont_pred.dat"
df_vert.to_csv(outputFILE_vert, header=None, index=None, sep=' ', float_format='%g')